# AWS Generative AI RAG Workshop
## Prompt Engineering with Amazon Bedrock

> *This notebook should work well with the **`Python 3`** kernel in SageMaker Studio*

---

In this demo notebook, we demonstrate how to use the [`boto3` Python SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) to work with [Amazon Bedrock](https://aws.amazon.com/bedrock/) Foundation Models.

---

## Prerequisites

Run the cells in this section to install the packages needed by the notebooks in this workshop. ⚠️ You will see pip dependency errors, you can safely ignore these errors. ⚠️

IGNORE ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

---

## Create the boto3 client

Interaction with the Bedrock API is done via the AWS SDK for Python: [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).

#### Converse API
**[Converse or ConverseStream API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html)** are unified structured text API action that allows you simplifying the invocations to Bedrock LLMs, using a universal syntax and message structured prompts for any of the supported model providers.


#### Use the default credential chain

If you are running this notebook from [Amazon Sagemaker Studio](https://aws.amazon.com/sagemaker/studio/) and your Sagemaker Studio [execution role](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) has permissions to access Bedrock you can just run the cells below as-is. This is also the case if you are running these notebooks from a computer whose default AWS credentials have access to Bedrock.



In [ ]:
import json
import os
import sys

import boto3
from utility import invoke_bedrock_model, invoke_bedrock_model_stream
from IPython.display import display, Markdown
import botocore
import json 

bedrock_runtime = boto3.client('bedrock-runtime')

# valid model id in the workshop environment
model_ids = [
    "anthropic.claude-3-sonnet-20240229-v1:0",
    "anthropic.claude-3-haiku-20240307-v1:0",
    "meta.llama3-70b-instruct-v1:0",
    "meta.llama3-8b-instruct-v1:0",
    "mistral.mistral-large-2402-v1:0",
]

---

## Lab 1 - Text Summarization in Bedrock

Picture yourself at a movie studio, or maybe a video streaming company. You get synopses all the time, and hardly have time to read them. In this Lab you will use Foundation Models (FMs) in Amazon Bedrock to summarize synopsis, so it is much easier to read and digest.

Here is a [Synopsis for the movie Whiplash](https://www.scriptreaderpro.com/wp-content/uploads/2019/07/Film-Synopsis-Example-Whiplash.pdf). We will use it below.



In [ ]:
prompt_data = """
Andrew Neiman, a young jazz student at the Shaffer Conservatory in New York, has one dream:
to go down in history as one of the world’s best drummers. He’s therefore thrilled when Terence
Fletcher, a famous conductor, invites him to join the conservatory’s Studio Band as a core
alternate drummer. Fletcher, however, turns out to be anything but an ordinary teacher. He’s a
sadistic tyrant and Andrew realizes just how much of one when he has a chair hurled at him for
failing to keep time.
At a jazz competition, Andrew misplaces the sheet music to “Whiplash,” meaning their core
drummer can’t play. Andrew, however, can—from memory—and after a first class performance,
Fletcher promotes him to core drummer. But Andrew’s joy won’t last long… In a typically
twisted move, Fletcher bumps Andrew back down to alternate drummer, putting a much lesstalented 
musician in his place. More determined than ever, Andrew breaks up with his girlfriend
and practices until his hands bleed. It pays off… After a grueling five-hour audition, during
which Fletcher kicks furniture and screams at him, Andrew earns back the core spot.
Andrew arrives late for another competition after his bus breaks down, hires a car, then realizes
he left his drumsticks at the car rental office. He races back, retrieves them, but on his way to
the theater, his car is broadsided by a semi. He crawls from the wreckage and runs the rest of
the way, finally arriving on stage bloody and injured. When he struggles to play, Fletcher cooly
dismisses him. Enraged, Andrew attacks Fletcher in front of the audience, which gets him
dismissed from the school.
Andrew files an ethics complaint against Shaffer Conservatory and learns that one of Fletcher’s
former students hanged himself due to his emotional and physical abuse. Andrew agrees to
testify as an anonymous witness and Fletcher is fired. Andrew gives up drumming and, months
later, stumbles upon Fletcher playing piano in a jazz club. They go for a drink, during which
Fletcher explains why he pushed his students so hard: so that they might become the next
Charlie Parker. In Fletcher’s eyes the greats like Parker wouldn’t be discouraged by anything.
He then invites Andrew to drum with his band at a jazz festival. Has Fletcher changed? Andrew
thinks so, and accepts.
On stage at the festival, Fletcher has two surprises for Andrew. One: he knows he testified
against him, and two: they’re starting with a piece Andrew doesn’t know and for which there’s
no sheet music. Unable to play, Andrew leaves the stage humiliated. But he returns, interrupts
Fletcher and cues the band, before launching into a breathtaking solo. Fletcher is taken aback,
but in that moment realizes the enormity of Andrew’s talent and begins to guide him. As
Andrew ends his solo, they share a smile and Fletcher cues the finale.

{INPUT}
"""

Next, we will construct the body and response to pass the prompt above to the Cohere Command text model.

In [ ]:
input_text = prompt_data.replace("{INPUT}", "Summarize the text above:")

for i in model_ids:
    response = invoke_bedrock_model(bedrock_runtime, i, input_text,
                                    max_tokens=400, temperature=0.75, top_p=0.9)
    print(f'### Model: {i}\n\n {response}')
    print ("-----\n")

## Refine the output

This is good, but there are ways to refine this result.

## Prompt engineering
Prompt engineering is a discipline focused on developing optimized prompts to efficiently apply language models to various tasks.

Try the same synopsis, but this time followed by "Summarize the text above in one sentence:".

In [ ]:
input_text = prompt_data.replace("{INPUT}", "Summarize the text above in one paragraph:")

for i in model_ids:
    response = invoke_bedrock_model(bedrock_runtime, i, input_text,
                                    max_tokens=400, temperature=0.75, top_p=0.9)
    print(f'### Model: {i}\n\n {response}')
    print("-----\n")

## Changing parameters.
Lets see what happens if we change the prompt parameters.

Below we are going to change the max_tokens to 50 lets see what happens.


In [ ]:
for i in model_ids:
    response = invoke_bedrock_model(bedrock_runtime, i, input_text,
                                    max_tokens=50, temperature=0.75, top_p=0.9)
    print(f'### Model: {i}\n\n {response}')
    print ("-----\n")

----
# Lab 2


# Text Generation Using Bedrock
If you are working with a script (whether for movies, television, game, etc), Foundation Models (FMs) can assist in a number of ways.

## Use FM to create dialogue
**FMs can generate dialogues for you.** - give the model a list of characters and a brief description of the scene, and let FMs generate the dialogues for a Comedy.

In [ ]:
streaming_ids = [
    "amazon.titan-text-express-v1",
    "anthropic.claude-3-sonnet-20240229-v1:0",
    "anthropic.claude-3-haiku-20240307-v1:0",
    "meta.llama3-70b-instruct-v1:0",
    "meta.llama3-8b-instruct-v1:0",
    "mistral.mistral-large-2402-v1:0",
]

In [ ]:
prompt = "as an expert script writer, write the dialogue between a husband and wife for a comedy scene."

In [ ]:
for i in streaming_ids:
    
    print(f'### Model: {i}')
    response = invoke_bedrock_model_stream(bedrock_runtime, i, prompt,
                                    max_tokens=400, temperature=0.75, top_p=0.9)
    print("\n\n-----\n")

## FMs can also help you brainstorm for plot ideas.

Ask Bedrock models to generate some plot ideas using the prompt below:

In [ ]:
prompt = """
as an award wining director, give me some ideas about a plot-point in a romantic comedy involving two young professionals who just met by chance in the supermarket after loosing track of one another after college.
"""

for i in streaming_ids:
    
    print(f'### Model: {i}')
    response = invoke_bedrock_model_stream(bedrock_runtime, i, prompt,
                                    max_tokens=400, temperature=0.75, top_p=0.9)
    print("\n\n-----\n")

---
# Lab 3
# Image Generation in Bedrock

Now that you have text generation , let's try image generation with Titan Image Generator  on Bedrock.

>Before we get started lets restart the kernel. 

### Image Prompting

Writing a good prompt can be somewhat of an art.

It is often difficult to predict whether a given prompt will yield a satisfactory result with a certain model.

However, there are certain templates that have been known to work.

Broadly, a prompt can be broken down into three pieces:
- Type of image (photograph/sketch/painting)
- Description of the content (subject/object/environment/scene/&c.), and
- Style of the image (realistic/artistic).

You can change each of the three parts individually to generate variations of an image.
Adjectives have been known to play a significant role in the image generation process.
Also, adding more details help in the generation process.

In order to generate a realistic image, you can use phrases such as:
- a photo of
- a photograph of
- realistic
- hyper realistic

To generate something more artistic, you can use phrases like:
- by Pablo Picasso
- oil painting by Rembrandt
- landscape art by Frederic Edwin Church
- pencil drawing by Albrecht Dürer

You can also combine different artists as well.
To generate artistic images by category, you can add the art category in the prompt such as
lion on a beach, abstract

Some other categories include:
- oil painting
- pencil drawing
- pop art
- digital art
- anime
- cartoon
- futurism
- watercolor
- manga


You can also include details such as lighting or camera lens such as:
- 35mm wide lens
- 85mm wide lens

or details about the framing:
- portrait
- landscape
- close up

Note that models can generate different images even if same prompt is given multiple times.

So, you can generate multiple images and select the image that suits your application best.

For more information on Amazon Titan Image Generator prompt engineering, see [Amazon Titan Image Generator Prompt Engineering Best Practices.](https://d2eo22ngex1n9g.cloudfront.net/Documentation/User+Guides/Titan/Amazon+Titan+Image+Generator+Prompt+Engineering+Guidelines.pdf)

In [ ]:
# Built-in libraries
import base64
import io
import json
import os
import sys

# External dependencies
import boto3
from PIL import Image
import botocore

boto3_bedrock = boto3.client('bedrock-runtime')


## Text to Image

In text-to-image mode, we provide a text description (prompt) of the image that should be generated.

What if we want to avoid specific content or stylistic choices? Because image generation models are typically trained from image descriptions, trying to directly specify what you don't want in the prompt (e.g. man without a beard) doesn't usually work well: it would be very unusual to describe an image by what it is not!

In the case of Amazon Titan Image Generator, we can specify a negative prompt to steer the model away from unwanted elements

For our attempt we will use "a photograph of an astronaut riding a horse" and "nsfw"

In [ ]:
prompt = "a photograph of an astronaut riding a horse"
negative_prompts = "nsfw"

The Amazon Bedrock InvokeModel provides access to Amazon Titan Image Generator by setting the right model ID, and returns a JSON response including a Base64 encoded string that represents the (PNG) image.

When making an InvokeModel request, we need to fill the body field with a JSON object that varies depending on the task (taskType) you wish to perform viz. text to image, image variation, inpainting or outpainting. The Amazon Titan models supports the following parameters:

- cfgscale - determines how much the final image reflects the prompt
- seed - a number used to initialize the generation, using the same seed with the same prompt + settings combination will produce the same results
- numberOfImages - the number of times the image is sampled and produced
- quality - determines the output image quality (standard or premium)

In [ ]:
# Create payload
body = json.dumps(
    {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt,                    # Required
            "negativeText": negative_prompts   # Optional
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,   # Range: 1 to 5 
            "quality": "standard",  # Options: standard or premium
            "height": 1024,        # Supported height list in the docs 
            "width": 1024,         # Supported width list in the docs
            "cfgScale": 8,       # Range: 1.0 (exclusive) to 10.0
            "seed": 1             # Range: 0 to 214783647
        }
    }
)

# Make model request
response = boto3_bedrock.invoke_model(
    body=body,
    modelId="amazon.titan-image-generator-v2:0",
    accept="application/json", 
    contentType="application/json"
)

# Process the image
response_body = json.loads(response.get("body").read())
img1_b64 = response_body["images"][0]

# Debug
print(f"Output: {img1_b64[0:80]}...")

By decoding our image string and loading it with an image processing library like [Pillow](https://pillow.readthedocs.io/en/stable/), we can display and manipulate the image


In [ ]:
img1 = Image.open(
    io.BytesIO(
        base64.decodebytes(
            bytes(img1_b64, "utf-8")
        )
    )
)
#print(os.getcwd())
img1.save("image_1.png")

# Display
img1

## Modify an Image

You can modify the image you just generated, this will further constrain the image generated. Let's change the prompt to **_Photograph of a astronaut riding a llama_**

In [ ]:
prompt = "a photograph of an astronaut riding a llama"
negative_prompts = "nsfw"

We will read the image file to a base64 object to pass to the model.

In [ ]:
def image_to_base64(img) -> str:
    """Converts a PIL Image or local image file path to a base64 string"""
    if isinstance(img, str):
        if os.path.isfile(img):
            print(f"Reading image from file: {img}")
            with open(img, "rb") as f:
                return base64.b64encode(f.read()).decode("utf-8")
        else:
            raise FileNotFoundError(f"File {img} does not exist")
    elif isinstance(img, Image.Image):
        buffer = io.BytesIO()
        img.save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue()).decode("utf-8")
    else:
        raise ValueError(f"Expected str (filename) or PIL Image. Got {type(img)}")

img1_b64 = image_to_base64(img1)
print(f"Input: {img1_b64[:80]}")

Lets pass this image to the model with our prompt.

In [ ]:
# Payload creation
body = json.dumps({
     "taskType": "IMAGE_VARIATION",
     "imageVariationParams": {
         "text": prompt,              # Optional
         "negativeText": negative_prompts,   # Optional
         "images": [img1_b64],               # One image is required
     },
     "imageGenerationConfig": {
         "numberOfImages": 1,
         "quality": "premium",
         "height": 1024,
         "width": 1024,
         "cfgScale": 8,
         "seed": 1
     }
 })

# Model invocation
response = boto3_bedrock.invoke_model(
    body=body,
    modelId="amazon.titan-image-generator-v2:0",
    accept="application/json", 
    contentType="application/json"
)

# Output processing
response_body = json.loads(response.get("body").read())
img2_b64 = response_body["images"][0]

# Debug
print(f"Output: {img2_b64[0:80]}...")

In [ ]:
img2 = Image.open(
    io.BytesIO(
        base64.decodebytes(
            bytes(img2_b64, "utf-8")
        )
    )
)
#print(os.getcwd())
img2.save("image_2.png")

# Display
img2

### Inpainting

Another way to modify images is by using inpainting.

Inpainting refers to the process of replacing a portion of an image with another image based on a textual prompt.

Let's define what we want to change in the image.

In [ ]:
prompt = "a photograph of an astronaut riding a camel"

Next we will pass the previous image to the model with the mask.

In [ ]:
# Payload creation
body = json.dumps({
    "taskType": "INPAINTING",
    "inPaintingParams": {
        "text": prompt,              # Optional
        "negativeText": negative_prompts,    # Optional
        "image": img2_b64,      # Required
        "maskPrompt": "llama",               # One of "maskImage" or "maskPrompt" is required
        # "maskImage": image_to_base64(mask),  # Input maskImage based on the values 0 (black) or 255 (white) only
    },                                                 
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "premium",
        "height": 1024,
        "width": 1024,
        "cfgScale": 8,
        "seed": 0
    }
})

# Model invocation
response = boto3_bedrock.invoke_model(
    body=body,
    modelId="amazon.titan-image-generator-v2:0",
    accept="application/json", 
    contentType="application/json"
)

# Output processing
response_body = json.loads(response.get("body").read())
img3_b64 = response_body["images"][0]
print(f"Output: {img3_b64[0:80]}...")

In [ ]:
inpaint = Image.open(
    io.BytesIO(
        base64.decodebytes(
            bytes(img3_b64, "utf-8")
        )
    )
)
inpaint.save("image_3.png")
inpaint

### Background removal

Using new functionality within [Titan Image Generation V2](https://aws.amazon.com/blogs/aws/amazon-titan-image-generator-v2-is-now-available-in-amazon-bedrock/) we can also remove the background. We will use the last image we generated as input. This is very helpful if we want to place an image on a different background.

In [ ]:
# Payload creation
body = json.dumps({
    # We use taskType to define what we want to do here
    "taskType": "BACKGROUND_REMOVAL",
    "backgroundRemovalParams": {
        "image": img3_b64,      # Required
    },                                                 
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "premium",
        "height": 1024,
        "width": 1024,
        "cfgScale": 8,
        "seed": 0
    }
})

# Model invocation
response = boto3_bedrock.invoke_model(
    body=body,
    modelId="amazon.titan-image-generator-v2:0",
    accept="application/json", 
    contentType="application/json"
)

# Output processing
response_body = json.loads(response.get("body").read())
img4_b64 = response_body["images"][0]
print(f"Output: {img4_b64[0:80]}...")

In [ ]:
removebg = Image.open(
    io.BytesIO(
        base64.decodebytes(
            bytes(img4_b64, "utf-8")
        )
    )
)
removebg.save("image_4.png")
removebg

### Color Guided Content

Another feature of [Titan Image V2](https://aws.amazon.com/blogs/aws/amazon-titan-image-generator-v2-is-now-available-in-amazon-bedrock/) is Color guided content. This allows you to guide the model as to the colors you would like to feature. Lets recreate the image above but give the model a color pallet we want it to stick to.

We will ask the model to use the following colors.

<p style="color:#caaaaa">Color 1 #caaaaa</p>
<p style="color:#ebb663">Color 2 #ebb663</p>





In [ ]:
# Payload creation
body = json.dumps({
            "taskType": "COLOR_GUIDED_GENERATION",
            "colorGuidedGenerationParams": {
                # We are using the same prompt as before
                "text": prompt,
                "negativeText": "bad quality, low res",
                "referenceImage": img2_b64,
                "colors": ["#caaaaa", "#ebb663"]
            },
            "imageGenerationConfig": {
                "numberOfImages": 1,
                "height": 1024,
                "width": 1024,
                "quality": "premium"
                "cfgScale": 8.0
            }
        })

# Model invocation
response = boto3_bedrock.invoke_model(
    body=body,
    modelId="amazon.titan-image-generator-v2:0",
    accept="application/json", 
    contentType="application/json"
)

# Output processing
response_body = json.loads(response.get("body").read())
img5_b64 = response_body["images"][0]
print(f"Output: {img5_b64[0:80]}...")

In [ ]:
colorgc = Image.open(
    io.BytesIO(
        base64.decodebytes(
            bytes(img5_b64, "utf-8")
        )
    )
)
colorgc.save("image_5.png")
colorgc

----
# Lab 4
# Question Answering in Bedrock

Question answering (QA) is an important task that involves extracting answers to factual queries posed in natural language. Foundation Models (FMs) like Amazon Titan, Anthropic Claude, and Cohere Command are trained on vast amounts of text. From that training, they develop the probability distributions that can predict next token/word in an answer sequence when given a question.

Despite how good these models have become, they are prone to **hallucination**, a phenomenon where a FMs generates false or inaccurate information. In the lab, we will experiment with different techniques to reduce hallucination and improve model response accuracy.

Lets ask some questions of the Amazon Titan Model:
- What is an Academy Award?
- In what year was the first Academy Awards ceremony?
- Who holds the record for the most Oscars won?
- Who nominates oscar nominees?

In [ ]:
model_ids = [
    "amazon.titan-text-express-v1",
]

In [ ]:
prompt_data = """
You are a question and answer chatbot. Please answer the following question.

{QUESTION}
"""

In [ ]:
input_text = prompt_data.replace("{QUESTION}", "What is an Academy Award?")

for i in model_ids:
    response = invoke_bedrock_model(bedrock_runtime, i, input_text,
                                    max_tokens=100, temperature=0.75, top_p=0.9)
    print(f'### Model: {i}\n\n {response}')
    print("-----\n")

So far so good. Now let's try a few question where the model may not answer correctly:
- What are the “Big Five” awards?
- In what year did ‘Forrest Gump’ win Best Picture?
- In what year was the first Oscar for Best Animated Feature awarded?

In [ ]:
input_text = prompt_data.replace("{QUESTION}", "What are the “Big Five” awards?")

for i in model_ids:
    response = invoke_bedrock_model(bedrock_runtime, i, input_text,
                                    max_tokens=100, temperature=0.75, top_p=0.9)
    print(f'### Model: {i}\n\n {response}')
    print("-----\n")

The model hallucinated, so let's try to fix it with prompt engineering.

In [ ]:
prompt_data = """
You are a question and answer chatbot. Please answer the following question. Say "I don't know" if you are not sure.

{QUESTION}
"""

In [ ]:
input_text = prompt_data.replace("{QUESTION}", "What are the “Big Five” awards?")

for i in model_ids:
    response = invoke_bedrock_model(bedrock_runtime, i, input_text,
                                    max_tokens=100, temperature=0.75, top_p=0.9)
    print(f'### Model: {i}\n\n {response}')
    print ("-----\n")

## In-Context Learning
Now let's help the model answer this correctly.

Following passage is copied from Wikipedia: [List of Big Five Academy Award winners and nominees.](https://en.wikipedia.org/wiki/List_of_Big_Five_Academy_Award_winners_and_nominees) Let's also feed this passage to Titan as context.

>At the Academy Awards, the so-called "Big Five" awards are those for Best Picture, Best Director, Best Actor, Best Actress, and Best Screenplay (either Best Original Screenplay or Best Adapted Screenplay).[1] As of the 94th Academy Awards (2021), a total of 43 films have been nominated in all five of these award categories. Only three films have won all five of these major awards: It Happened One Night (1934), One Flew Over the Cuckoo's Nest (1975), and The Silence of the Lambs (1991). Eight films failed to win any of the five major awards after being nominated.

In [ ]:
context = """
At the Academy Awards, the so-called "Big Five" awards are those for Best Picture, 
Best Director, Best Actor, Best Actress, and Best Screenplay (either Best Original 
Screenplay or Best Adapted Screenplay).[1] As of the 94th Academy Awards (2021), a 
total of 43 films have been nominated in all five of these award categories. Only 
three films have won all five of these major awards: It Happened One Night (1934), 
One Flew Over the Cuckoo's Nest (1975), and The Silence of the Lambs (1991). Eight 
films failed to win any of the five major awards after being nominated.
"""

In [ ]:
prompt_data = """
You are a question and answer chatbot. Please answer the following question only use the context below. Say "I don't know" if you are not sure.

{CONTEXT}

{QUESTION}
"""

In [ ]:
input_text = prompt_data.replace("{QUESTION}", "What are the “Big Five” awards?")
input_text = input_text.replace("{CONTEXT}", context)

for i in model_ids:
    response = invoke_bedrock_model(bedrock_runtime, i, input_text,
                                    max_tokens=100, temperature=0.75, top_p=0.9)
    print(f'### Model: {i}\n\n {response}')
    print ("-----\n")

>Thank you, you have completed this section of the workshop.